In [3]:
!pip install scipy --upgrade --quiet
import scipy
print("SciPy version:", scipy.__version__)


SciPy version: 1.16.3



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2  # Smaller than EfficientNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print("TensorFlow:", tf.__version__)


TensorFlow: 2.20.0


In [5]:
IMG_SIZE = 180

datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)
datagen_test = ImageDataGenerator(rescale=1./255)

train_set = datagen_train.flow_from_directory(
    "../dataset/pokemon/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=32
)

test_set = datagen_test.flow_from_directory(
    "../dataset/pokemon/validation",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=32
)

print("Train samples:", train_set.samples)
print("Val samples:", test_set.samples)
print("Classes:", len(train_set.class_indices))


Found 4978 images belonging to 150 classes.
Found 1842 images belonging to 150 classes.
Train samples: 4978
Val samples: 1842
Classes: 150


In [6]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(len(train_set.class_indices), activation='softmax')(x)

model = Model(inputs, outputs)
model.summary()


C:\Users\91916\AppData\Local\Temp\ipykernel_20832\2035857503.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 6, 6, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 150)                 │          19,350 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,441,302 (9.31 MB)

 Trainable params: 183,318 (716.09 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [7]:
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=1e-7, verbose=1)
]


In [8]:
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_set,
    validation_data=test_set,
    epochs=20,
    callbacks=callbacks
)


Epoch 1/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 130s 800ms/step - accuracy: 0.0237 - loss: 4.9413 - val_accuracy: 0.0782 - val_loss: 4.6224 - learning_rate: 0.0010
Epoch 2/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 137s 876ms/step - accuracy: 0.1107 - loss: 4.2350 - val_accuracy: 0.2557 - val_loss: 3.4823 - learning_rate: 0.0010
Epoch 3/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 158s 1s/step - accuracy: 0.2290 - loss: 3.4090 - val_accuracy: 0.4359 - val_loss: 2.6587 - learning_rate: 0.0010
Epoch 4/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 144s 925ms/step - accuracy: 0.3146 - loss: 2.8932 - val_accuracy: 0.5201 - val_loss: 2.1782 - learning_rate: 0.0010
Epoch 5/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 147s 938ms/step - accuracy: 0.3777 - loss: 2.5336 - val_accuracy: 0.5429 - val_loss: 1.9967 - learning_rate: 0.0010
Epoch 6/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 124s 796ms/step - accuracy: 0.4257 - loss: 2.3010 - val_accuracy: 0.6021 - val_loss: 1.7794 - learning_rate: 0.0010
Epoch 7/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 89s 568ms/step - accuracy: 

In [9]:
# Best validation accuracy during training
best_val_acc = max(history.history['val_accuracy'])
print("Best val_accuracy during training:", best_val_acc)

# Final evaluation on validation set
val_loss, val_acc = model.evaluate(test_set)
print("Final validation loss:", val_loss)
print("Final validation accuracy:", val_acc)

# Also print training accuracy for comparison
best_train_acc = max(history.history['accuracy'])
print("Best training accuracy:", best_train_acc)


Best val_accuracy during training: 0.709011971950531
58/58 ━━━━━━━━━━━━━━━━━━━━ 17s 297ms/step - accuracy: 0.7090 - loss: 1.1410
Final validation loss: 1.1409742832183838
Final validation accuracy: 0.709011971950531
Best training accuracy: 0.617918848991394


In [10]:
model.save("pokemon_mobilenetv2.keras")

In [12]:
# Load the saved Keras model (or use `model` directly if it's still in memory)
keras_model = tf.keras.models.load_model("pokemon_mobilenetv2.keras")

# Create converter from Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflite_model = converter.convert()

# Save the TFLite model to disk
with open("pokemon_classifier.tflite", "wb") as f:
    f.write(tflite_model)

print("Saved TFLite model as pokemon_mobilenetv2.tflite")


INFO:tensorflow:Assets written to: C:\Users\91916\AppData\Local\Temp\tmpgombl47h\assets


INFO:tensorflow:Assets written to: C:\Users\91916\AppData\Local\Temp\tmpgombl47h\assets


Saved artifact at 'C:\Users\91916\AppData\Local\Temp\tmpgombl47h'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 150), dtype=tf.float32, name=None)
Captures:
  2497318513232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2497032230608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2497032233872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495664162384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2497318515536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495955679184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495955679376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495955679568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495955680528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2495955680912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  24